# 03 EEG Preprocess RANSAC

## Overview
This notebook identifies **bad (noisy) EEG channels** across all recording sessions using the RANSAC algorithm from PyPREP. 

**Purpose:**
- Automatically detect malfunctioning or noisy electrodes that would contaminate the EEG signal
- Identify channels with poor contact, excessive noise, or artifacts
- Generate a summary report showing which channels are problematic across sessions

**What it does:**
1. Loads preprocessed raw EEG data from Notebook 02 (`session_XX-EEG-raw.pkl`)
2. Applies high-pass filtering (1 Hz) to remove slow drifts
3. Uses RANSAC (Random Sample Consensus) to detect bad channels by:
   - Comparing each channel against predictions from neighboring channels
   - Identifying channels that deviate significantly from expected patterns
4. Generates summary statistics showing which electrodes are frequently bad across sessions

**Output:**
- DataFrame showing bad channels per session
- Counter showing how often each electrode is marked as bad
- This information is used in Notebook 04 to exclude bad channels from further analysis

**Next step:** Based on the results, you'll manually remove consistently bad channels before running Notebook 04 (ICA preprocessing).

**Code Attribution:**
- Original EEG preprocessing code adapted from: Chiossi, F., Mayer, S., & Ou, C. (2024). MobileHCI 2024 Papers - Submission 7226.
- OSF Repository: https://osf.io/fncj4/overview (Created: Sep 11, 2023)
- License: GNU General Public License (GPL) 3.0
- Code has been modified for this study's session-based structure and experimental design.

## 1. Import Libraries

In [11]:
import pandas as pd
import numpy as np
import mne
import pyprep
from collections import Counter

## 2. Load Session Mapping

In [12]:
# All 64 EEG channels (excluding Time, TimeLsl, and accelerometer data)
chan_names = ['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz', 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'Iz', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4', 'AF8']

# Load session mapping
df_sessions = pd.read_csv('./session_mapping.csv')
df_matched = df_sessions[df_sessions['eeg_file'] != 'NO MATCH'].copy()

# Find all available preprocessed pickle files
from pathlib import Path
import re

preprocessed_dir = Path('./preprocessed')
available_sessions = []

for pkl_file in sorted(preprocessed_dir.glob('session_*-EEG-raw.pkl')):
    # Extract session number from filename (e.g., session_00-EEG-raw.pkl -> 0)
    match = re.search(r'session_(\d+)-EEG-raw\.pkl', pkl_file.name)
    if match:
        session_id = int(match.group(1))
        available_sessions.append(session_id)

session_ids = sorted(available_sessions)

print(f"Channel names: {len(chan_names)}")
print(f"Total matched sessions: {len(df_matched)}")
print(f"Available preprocessed sessions: {len(session_ids)}")
print(f"Session IDs: {session_ids}")

Channel names: 64
Total matched sessions: 21
Available preprocessed sessions: 21
Session IDs: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


# 3. Test Bad Channels after Preprocessing the Data
### Once bad channels have been identified, run the preprocessing above again without them. If the bad channel detection fails for one participant, remove the participant.

In [13]:
def eeg_getbads(session_id, chan_names, n_jobs=10):
    """Identify bad channels for a session."""
    
    dfEEG = pd.read_pickle(f"./preprocessed/session_{session_id:02d}-EEG-raw.pkl")
    
    # Convert to float64 and handle NaN/Inf values
    eeg_data = dfEEG[chan_names].values.T / 1000000
    eeg_data = np.nan_to_num(eeg_data, nan=0.0, posinf=0.0, neginf=0.0)
    eeg_data = eeg_data.astype(np.float64)
    
    info = mne.create_info(ch_names=chan_names, sfreq=500, ch_types='eeg', verbose=False)
    info.set_montage('standard_1020')
    info['subject_info'] = {"id": session_id}
    info['subject_info'] = {"his_id": str(session_id)}
    raw = mne.io.RawArray(eeg_data, info, verbose=False)
    
    raw = raw.filter(l_freq=1, h_freq=None, fir_design='firwin2', verbose=False, n_jobs=n_jobs) 

    # Identify bad channels
    try:
        nc = pyprep.find_noisy_channels.NoisyChannels(raw, random_state=42)
        nc.find_all_bads()
        return nc.get_bads(verbose=False)
    except Exception as e:
        print(f"  ⚠️ Error during RANSAC: {e}")
        return []


In [19]:
lst = []

# Try to load existing dfBad if it exists in the namespace
try:
    if 'dfBad' in dir() and len(dfBad) > 0:
        print(f"✓ Found existing dfBad with {len(dfBad)} sessions")
        print("  Converting to lst format...")
        for _, row in dfBad.iterrows():
            lst.append([row['SessionID'], row['Electrodes']])
        print(f"  Loaded {len(lst)} sessions from existing dfBad")
except:
    print("No existing dfBad found, will process from scratch")


In [20]:
for i, session_id in enumerate(session_ids):
    # Check if RANSAC results already exist
    result_path = preprocessed_dir / f"session_{session_id:02d}-bad-channels.pkl"
    if result_path.exists():
        print(f"[{i+1}/{len(session_ids)}] Session {session_id}: ⏭️  Skipping (already analyzed)")
        # Load existing results
        x = pd.read_pickle(result_path)
        lst.append([session_id, x])
        continue

    print(f"[{i+1}/{len(session_ids)}] Processing Session {session_id}...")
    x = eeg_getbads(session_id, chan_names)
    lst.append([session_id, x])
    print(f"  Found bad channels: {x}")

    # Save RANSAC results
    pd.to_pickle(x, result_path)
    print(f"  Saved: {result_path.name}")


[1/21] Processing Session 0...
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1651 samples (3.302 s)
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1651 samples (3.302 s)


Executing RANSAC
This may take a while, so be patient...
Executing RANSAC
This may take a while, so be patient...


100%|██████████|  : 761/761 [03:31<00:00,    3.60it/s]


RANSAC done!


  Found bad channels: ['FT7', np.str_('POz'), np.str_('AF8'), np.str_('F4'), np.str_('P9'), np.str_('F7'), np.str_('F3')]
  Saved: session_00-bad-channels.pkl
[2/21] Processing Session 1...
  Saved: session_00-bad-channels.pkl
[2/21] Processing Session 1...
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1651 samples (3.302 s)

Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 d

100%|██████████|  : 421/421 [01:42<00:00,    4.11it/s]

...


RANSAC done!
  Found bad channels: [np.str_('T8'), 'C3', 'F9', np.str_('CP6'), 'Fz', np.str_('PO3'), np.str_('CP3'), 'FC5', np.str_('F3'), np.str_('P10'), np.str_('TP7')]
  Saved: session_18-bad-channels.pkl
[20/21] Processing Session 19...
  Found bad channels: [np.str_('T8'), 'C3', 'F9', np.str_('CP6'), 'Fz', np.str_('PO3'), np.str_('CP3'), 'FC5', np.str_('F3'), np.str_('P10'), np.str_('TP7')]
  Saved: session_18-bad-channels.pkl
[20/21] Processing Session 19...
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1651 samples (3.302 s)

Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causa

100%|██████████|  : 148/148 [00:34<00:00,    4.27it/s]
100%|██████████|  : 148/148 [00:34<00:00,    4.27it/s]



RANSAC done!
  Found bad channels: ['C3', 'F9', 'CP6', 'Iz', 'Fz', np.str_('PO3'), np.str_('CP3'), 'FC5', np.str_('F3'), np.str_('P10'), np.str_('TP7')]
  Saved: session_19-bad-channels.pkl
[21/21] Processing Session 20...
  Found bad channels: ['C3', 'F9', 'CP6', 'Iz', 'Fz', np.str_('PO3'), np.str_('CP3'), 'FC5', np.str_('F3'), np.str_('P10'), np.str_('TP7')]
  Saved: session_19-bad-channels.pkl
[21/21] Processing Session 20...
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1651 samples (3.302 s)

Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-d

100%|██████████|  : 187/187 [00:40<00:00,    4.63it/s]




RANSAC done!
  Found bad channels: ['C3', 'F9', 'Iz', 'Fz', 'F10', 'AF7', 'FC5', 'F3']
  Saved: session_20-bad-channels.pkl
  Found bad channels: ['C3', 'F9', 'Iz', 'Fz', 'F10', 'AF7', 'FC5', 'F3']
  Saved: session_20-bad-channels.pkl


In [21]:
dfBad = pd.DataFrame(lst)

if len(dfBad) > 0:
    dfBad.columns = ['SessionID', 'Electrodes']

    dfBadCounter = pd.DataFrame.from_dict(Counter(np.concatenate(dfBad.Electrodes.values).ravel()), orient='index').reset_index()
    dfBadCounter = dfBadCounter.rename(columns={'index':'Electrode', 0:'Count'})
    dfBadCounter = dfBadCounter.sort_values("Count")
    dfBadCounter
else:
    print("⚠️ No sessions were processed. All were skipped or lst is empty.")

In [22]:
dfBad

,SessionID,Electrodes
0,0,"[FT7, POz, AF8, F4, P9, F7, F3]"
1,1,"[C3, F9, CP5, Pz, F10, O1, FC5, C5, C1]"
2,2,"[AF3, P5, FC2, POz, C2, Fp1, F6, FC3, PO8, PO3..."
3,3,"[Cz, C3, F9, Iz, Fz, Fp1, FC6, F10, AF7, FC5, ..."
4,4,"[C3, F9, CP6, CP5, F10]"
5,5,"[C3, F9, CP6, CP5, F10]"
6,6,"[T8, AF3, C3, Fp1, FC6, FC1, FT8, O1, FC5]"
7,7,"[Cz, P1, Fp2, Iz]"
8,8,"[F9, Iz, CP5, Fp1, F10, AF7, CP3, P10, F3, TP7]"
9,9,"[AF3, C3, F9, P1, CP5, Fp1, FC6, F10, O2, P10,..."
